In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
covidCompleto = pd.read_csv('/kaggle/input/corona-virus-report/covid_19_clean_complete.csv')
display(covidCompleto.head())

# O primeiro passo da exploração de dados é obter informações gerais, deve-se
# importar o arquivo “covid_19_clean_complete.csv” e obter as seguintes informações

#a. Quantidade de linhas e colunas
covidCompleto.shape

#b. Quais são as colunas
covidCompleto.columns

#c. O tipo de dado de cada coluna
covidCompleto.info()

#d. A coluna de datas deve ser transformada de “Object” para “datetime64”, utilizando o comando “pd.to_datetime( )”
covidCompleto.Date = pd.to_datetime(covidCompleto.Date)

#e. Informações estatísticas sobre o banco de dados
covidCompleto.info()

#f. Quais colunas apresentam NaN
covidCompleto.isnull().sum(axis=0)


In [4]:
# Crie um dataframe que contenha apenas as 5 províncias da China que mais registraram casos 

#a. Descubra qual o nome das províncias chinesas contidas no dataset
covidCompletoChina_Provinces = covidCompleto[covidCompleto["Country/Region"] == "China"]["Province/State"]
display(covidCompletoChina_Provinces.unique())

#b. Retire do banco de dados apenas as informações dessas províncias
covidCompletoChina = covidCompleto[covidCompleto["Country/Region"] == "China"]
display(covidCompletoChina)

#c. Tome apenas as informações das features: “Confirmed”, “Active”, “Deaths”, “Recovered”
covidCompletoChina = covidCompletoChina.loc[:,['Province/State','Confirmed','Deaths','Recovered','Active']]

#d. Agrupe o novo dataframe a partir dessas províncias. Qual a função de agrupamento que deve ser usada?
#e. Produza um dataframe contendo as 5 regiões com o maior número de casos confirmados
covidCompletoChina = covidCompletoChina.groupby("Province/State").sum().sort_values("Confirmed", ascending = False)
display(covidCompletoChina.head(5))



In [5]:
#a. Elabore uma função que tenha uma linha do banco de dados como argumento e, se a coluna “Province/State” não 
# for valor faltante, concatenar seu valor com coluna “Country/Region”
    #i. Utilize o método “pandas.notna” para averiguar de “Province/State” é valor faltante
def verifyProvinces(linha):
    if pd.notna(linha["Province/State"]):
        linha["Country/Region"] = linha["Country/Region"] + " - " + linha["Province/State"]
    return linha

#b. Faça uma cópia do banco de dados para poder retirar informações sem perder o banco de dados original
covidProvinces = covidCompleto.copy()

#c. Aplique, por meio do método “apply”, a função criada no item a
covidProvinces = covidProvinces.apply(verifyProvinces, axis = 1)

#d. Exclua a coluna “Province/State” do novo dataframe
covidProvinces = covidProvinces.drop(["Province/State"], axis = 1)
covidProvinces.head(20)



    
    


In [6]:
#4. Importe o arquivo "world meter data.csv" e, a partir de seus dados de população e
# continentes, faça um ranking de maior número de mortes por milhão de habitantes
# entre os continentes.

#a. Importar o banco de dados "worldmeter_data.csv"
worldMeter = pd.read_csv("/kaggle/input/corona-virus-report/worldometer_data.csv")
worldMeter.head()

#b. Tome apenas as informações de população, país e continentes deste banco de dados
worldMeter_Localizacao = worldMeter.loc[:,["Country/Region", "Continent", "Population"]]
display(worldMeter_Localizacao.head())


#c. Agrupe o “covid_19_clean_complete” por países. Deve-se usar o dataframe do item “c)” ou o original?
covidCompleto_Reduzido = covidCompleto[['Country/Region','Deaths']].groupby('Country/Region').sum()
display(covidCompleto_Reduzido)

#d. Mescle os dois dataframes, a partir dos países, contendo apenas as informações de mortes, população e continente
covidMundo = covidCompleto_Reduzido.merge(worldMeter[['Country/Region','Population','Continent']], on = 'Country/Region')
display(covidMundo)

#e. Agrupe o novo dataset agora pelo continente
covidMundo = covidMundo.groupby("Continent", as_index = False).sum()
display(covidMundo)

#f. Crie uma nova coluna contendo o número de mortes por milhão dentro dos
# continentes (Utilize a fórmula: (mortes/população)*10^6)
covidMundo["Mortes por milhão"] = (covidMundo['Deaths']/covidMundo['Population'])*10**6
display(covidMundo)

#g. Faça um ranking do número de mortes por milhão de habitantes entre os continentes
covidMundo = covidMundo.sort_values(by = "Mortes por milhão", ascending = False)
covidMundo["Ranking"] = [x for x in range(1, len(covidMundo) + 1)] 
covidMundo_ = covidMundo.set_index("Ranking")
display(covidMundo_)




**Visualização de dados**
.
.
.
.
.

In [7]:
#Criar gráfico linechart que indique a progressão no tempo das mortes nas regiões:
# Américas, Eastern Mediterran e Europe (OBS: Utilize apenas o arquivo “covid_19_clean_complete.csv”)

#a. Importar bibliotecas matplotlib e seaborn. Para retirar a necessidade de criar
# arquivos para os gráficos, digitar comando “%matplotlib inline”
# (importação realizada na primeira célula de código)

#b. Realizar agrupamento utilizando as colunas “Date” e “WHO Region”, utilizando
# “sum” como função de agregação
covidAgrupGraf = covidCompleto.groupby(by = ["Date", "WHO Region"]).sum().reset_index(level = "Date")
display(covidAgrupGraf)

#c. Plotar gráficos que contenham as progressões das regiões exigidas
plt.figure(figsize = (20, 12))
plt.plot("Date", "Deaths", data = covidAgrupGraf.loc[covidAgrupGraf.index == "Americas"], c = "r")
plt.plot("Date", "Deaths", data = covidAgrupGraf.loc[covidAgrupGraf.index == "Eastern Mediterranean"], c = "g")
plt.plot("Date", "Deaths", data = covidAgrupGraf.loc[covidAgrupGraf.index == "Europe"], c = "b")

#d. Adicionar nomes para eixos, título do gráfico, grid e legendas para as linhas
sns.set_style("darkgrid")
plt.title("Progressão de mortes por Covid em diferentes regiões\n", fontsize=20)
plt.legend(["Americas","Eastern Mediterranean","Europe"], fontsize = 20)
plt.xlabel("Date", fontsize = 20)
plt.ylabel("Deaths", fontsize = 20)

#e. Analisar o gráfico de acordo com as perguntas feitas
# Eastern Mediterranean apresentou uma progressão mais baixa em relação as regiões analisadas.
# Europe apresentou uma explosão na progressão de mortes, mas a partir da segunda metade do ano reduziu a taxa que apresentava antes.
# Americas apresentou uma taxa crescente a partir de Abril de 2020.


In [8]:
#Agora deve-se produzir um gráfico bar chart, indicando o número de mortes em cada continente

#a. Editar o tamanho do gráfico, como sugestão, coloque o tamanho (10,6)
plt.figure(figsize = (20,12))
plt.rcParams['xtick.labelsize'] = 15
plt.rcParams['ytick.labelsize'] = 15

#b. Plotar gráfico bar chart das mortes nos continentes
plt.bar("Continent", "Deaths", data = covidMundo_)

#c. editar título e nome dos eixos

plt.title("Mortes por Covid-19 nos continentes\n", fontsize = 20)
plt.xlabel("Continent", fontsize = 20)
plt.ylabel("Deaths", fontsize = 20)

#d. Analisar o gráfico de acordo com as perguntas feitas
# Continentes com mais mortes: Europa e América do Sul
# Continente com menos mortes: Oceania
# Hipóteses: tempo para contato com o vírus, sistema de saúde, medidas de segurança adotadas, conscientização da população 






In [55]:
#Descobrir se há relação entre o número de mortes por milhão e a quantidade de 
# habitantes, nos 3 continentes com maior média de mortes por milhão de habitantes,
# por meio de scatter plot

#a. Agrupar o dataframe pela coluna “Continent”, retirar apenas os dados de
# "Deaths/1 M pop", utilizar “mean” como função de agregação e ordenar para
# descobrir os 3 continentes com maior valor
continenteMorteMilhao = worldMeter[["Continent", "Deaths/1M pop"]].groupby("Continent", as_index = False).mean().sort_values("Deaths/1M pop", 
                                                                                                                            ascending = False)                                                                                                            
display(continenteMorteMilhao)

#b. Obter apenas os dados do dataframe, correspondentes a esses continentes
mortesRelacPop = worldMeter.loc[(worldMeter["Continent"] == "South America")|(worldMeter["Continent"] == "Europe")|
                                (worldMeter["Continent"] == "North America"),["Continent", "Population", "Deaths/1M pop"]]
display(mortesRelacPop)

#c. Criar Scatterplot sendo o eixo x mostrando as mortes por milhão, o eixo y, a
# população total e representar os 3 continentes com cores diferentes
plt.figure(figsize = (20,12))
sns.scatterplot(x = "Population", y = "Deaths/1M pop", hue = "Continent", data = mortesRelacPop)





In [54]:
# a. Criar uma variável para armazenar os valores de correlação
correlacao = worldMeter[["Population","Deaths/1M pop"]]

#b. Criar um heatmap, com título, utilizando a variável criada no item anterior
plt.figure(figsize = (20,12))
plt.title("Correlação entre colunas\n", fontsize = 20)
sns.heatmap(correlacao.corr(), annot = True, vmin = -1, cmap = "viridis")

#c. Escolher duas features para analisar seus valores de correlação (Sugestão: “Population”, “Teste/1M pop”)